<a href="https://colab.research.google.com/github/HebahAlshamlan/CITC_Task/blob/master/CITC_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.utils import shuffle

pd.set_option('max_colwidth',400)

In [ ]:
dataset = pd.read_csv('cars.csv')
print(dataset.shape)

dataset.head()

(301, 8)


,car_name,year,mileage,fuel_type,seller_type,transmission_type,previous_owners,selling_price
0,ritz,2014.0,27000,Petrol,Dealer,Manual,0,3.35
1,sx4,2013.0,43000,Diesel,Dealer,Manual,0,4.75
2,ciaz,2017.0,6900,Petrol,Dealer,Manual,0,7.25
3,wagon r,2011.0,5200,Petrol,Dealer,Manual,0,2.85
4,swift,2014.0,42450,Diesel,Dealer,Manual,0,4.60


In [ ]:
dataset.dtypes


car_name              object
year                 float64
mileage                int64
fuel_type             object
seller_type           object
transmission_type     object
previous_owners        int64
selling_price        float64
dtype: object

In [ ]:
# Find what the duplicate were
dataset.drop_duplicates(subset=None, inplace=True)
dataset.shape

(299, 8)

Exploratory Data Analysis

In [ ]:
print(dataset.selling_price.mean())
print(dataset.selling_price.median())
print(dataset.selling_price.max())
print(dataset.selling_price.min())

4.57063758389262
3.505
35.0
0.1


Missing values


In [ ]:
dataset.isna().any()

car_name              True
year                  True
mileage              False
fuel_type            False
seller_type          False
transmission_type    False
previous_owners      False
selling_price         True
dtype: bool